In [30]:
import subprocess
from subprocess import call
import pandas as pd
import os
import shutil
import gzip
import json
# import sys


def uncompress_gzip(file_name, new_name=None, delete=True):
    # Read the stream and write that stream to a new file:
    in_file = gzip.open(file_name, 'rb')
    if new_name is None:
        out_file = open(file_name.strip('.gz'), 'wb')
    else:
        out_file = open(new_name, 'wb')
    out_file.write(in_file.read())
    in_file.close()
    out_file.close()
    if delete:
        os.remove(file_name)
        
def execute(comando, doitlive=False, input_to_use=None):
    # result = subprocess.run(['ls', '-l'], stdout=subprocess.PIPE)
    comando = comando.split(' ')

    if doitlive:
        popen = subprocess.Popen(comando, stdout=subprocess.PIPE, universal_newlines=True)
        to_return = popen.stdout.read()
        for line in to_return:
            print(line, end='')
        popen.stdout.close()
        return_code = popen.wait()
        if return_code:
            raise subprocess.CalledProcessError(return_code, comando)
    else:
        if input_to_use is not None:
            input_to_use = input_to_use.ecode('utf-8')
        result = subprocess.run(comando, stdout=subprocess.PIPE, stderr=subprocess.PIPE, input=input_to_use)
        to_return = result.stdout.decode('utf-8')
        print(to_return)
    return to_return.strip('\n')

In [31]:
dataset_name = 'BRCA_all'

In [32]:
# Download from manifest
# manifest = "gdc_manifest_20171221_005438.txt"
manifest = "gdc_medifest_20171221_005438.txt"
command = "./gdc-client download -m "+manifest
print('This download may take a while, please be patient.')
execute(command, doitlive=True)
print('All files in manifest were downloaded, probably. To be sure, check the output of the gdc-client.')

This download may take a while, please be patient.
  0% [                                            ] ETA:  --:--:--   0.00  B/s 
100% [#############################################] ETA:  0:00:00   0.68  B/s 
100% [#############################################] Time: 0:00:01   0.68  B/s 
  0% [                                            ] ETA:  --:--:--   0.00  B/s 
100% [#############################################] ETA:  0:00:00   0.84  B/s 
100% [#############################################] Time: 0:00:01   0.84  B/s 
  0% [                                            ] ETA:  --:--:--   0.00  B/s 
100% [#############################################] ETA:  0:00:00   0.65  B/s 
100% [#############################################] Time: 0:00:01   0.65  B/s 
  0% [                                            ] ETA:  --:--:--   0.00  B/s 
100% [#############################################] ETA:  0:00:00   0.39  B/s 
100% [#############################################] Time: 0:00:02   

In [33]:
# Read manifest and create a dictionary of useful data
dfest = pd.read_table(manifest)

# Parse metadata
metadata_file = "metadata.cart.2017-12-21T21_41_22.870798.json"
meta_data = json.load(open(metadata_file))

# Make a dictionary to map file names to TCGA unique ID's
#   # if we wanted to have only one sample per patient per "phenotype" we could use this:
#   # meta_data[0]['cases'][0]['samples'][0]['submitter_id']
#   # But since we want a unique ID for each HTSeq file (some patient/phenotypes will have multiple vials/replicates):
#   # meta_data[0]['cases'][0]['samples'][0]['portions'][0]['analytes'][0]['aliquots'][0]['submitter_id']
#   # Read more hre: https://wiki.nci.nih.gov/display/TCGA/Understanding+TCGA+Biospecimen+IDs
name_id_dict = {}
for i in range(len(meta_data)):
    file_name = meta_data[i]['file_name']
    unique_id = meta_data[i]['cases'][0]['samples'][0]['portions'][0]['analytes'][0]['aliquots'][0]['submitter_id']
    name_id_dict[file_name] = unique_id

In [34]:
# pwd = os.path.dirname(__file__)
pwd = execute('pwd')
destination = os.path.join(pwd, 'raw_files')
if not os.path.isdir(destination):
    os.mkdir(destination)

for d, f in zip(dfest['id'], dfest['filename']):
    shutil.copy(os.path.join(d, f), destination)  # Move the downloaded files to a folder
    shutil.rmtree(d)  # Remove those files/folders from current directory
    # "decompress" and remove gz files
    uncompress_gzip(os.path.join(destination, f), new_name=os.path.join(destination, name_id_dict[f]+'.htseq.counts'))
print('All files were moved and "decompressed" successfully.')

/Users/edjuaro/GoogleDrive/tcga/step_1

All files were moved and "decompressed" successfully.


# From a list of HTSeq.counts files to a gct:

## Make sample info file

In [35]:
class_dict ={
    '01':'Tumor',
    '02':'Tumor',
    '03':'Tumor',
    '04':'Tumor',
    '05':'Tumor',
    '06':'Tumor',
    '07':'Tumor',
    '08':'Tumor',
    '09':'Tumor',
    '10':'Normal',
    '11':'Normal',
    '12':'Normal',
    '13':'Normal',
    '14':'Normal',
    '15':'Normal',
    '16':'Normal',
    '17':'Normal',
    '18':'Normal',
    '19':'Normal',
}

In [36]:
name = 'TCGA_'+dataset_name+'.txt'
file = open(name, 'w')
file.write('File\tClass\tSample_Name\n')
for f in dfest['filename']:
    
    #TODO: add a filter here for the types of samples we want. I am using all "0x"and "1x" samples... 
    #but presumably we only want "01" and "11" but then we should remove those from the directory (moved them to "unused")
    
#     if name_id_dict[f][13:15] == '01':
#         # file.write('\t'.join([name_id_dict[f]+'.htseq',class_dict[name_id_dict[f][17:19]] ,name_id_dict[f]]))
#         file.write('\t'.join([name_id_dict[f]+'.htseq','Tumor', name_id_dict[f]]))
#         file.write('\n')
#         print(name_id_dict[f])
#     if name_id_dict[f][13:15] == '11':
#         file.write('\t'.join([name_id_dict[f]+'.htseq','Normal', name_id_dict[f]]))
#         file.write('\n')
#         print(name_id_dict[f])
    if name_id_dict[f][13] == '0':
        # file.write('\t'.join([name_id_dict[f]+'.htseq',class_dict[name_id_dict[f][17:19]] ,name_id_dict[f]]))
        file.write('\t'.join([name_id_dict[f]+'.htseq','Tumor', name_id_dict[f]]))
        file.write('\n')
        print(name_id_dict[f])
    if name_id_dict[f][13] == '1':
        file.write('\t'.join([name_id_dict[f]+'.htseq','Normal', name_id_dict[f]]))
        file.write('\n')
        print(name_id_dict[f])
file.close()

TCGA-A8-A07U-01A-11R-A034-07
TCGA-E2-A15E-06A-11R-A12D-07
TCGA-D8-A1JM-01A-11R-A13Q-07
TCGA-AN-A0AT-01A-11R-A034-07
TCGA-EW-A3E8-01B-11R-A24H-07
TCGA-A8-A085-01A-11R-A00Z-07
TCGA-AO-A1KQ-01A-11R-A13Q-07
TCGA-BH-A0E0-01A-11R-A056-07
TCGA-D8-A1JS-01A-11R-A13Q-07
TCGA-E2-A2P5-01A-11R-A19W-07
TCGA-D8-A1J9-01A-11R-A13Q-07
TCGA-BH-A0H5-11A-62R-A115-07
TCGA-E2-A56Z-01A-12R-A29R-07
TCGA-OL-A5RZ-01A-11R-A28M-07
TCGA-BH-A1EO-01A-11R-A137-07
TCGA-BH-A0HQ-01A-11R-A034-07
TCGA-B6-A0IO-01A-11R-A034-07
TCGA-D8-A1XO-01A-11R-A14M-07
TCGA-BH-A28Q-01A-11R-A16F-07
TCGA-AN-A0FT-01A-11R-A034-07
TCGA-BH-A0HA-11A-31R-A12P-07
TCGA-AO-A0JJ-01A-11R-A056-07
TCGA-AO-A129-01A-21R-A10J-07
TCGA-E9-A54Y-01A-11R-A466-07
TCGA-V7-A7HQ-01A-11R-A33J-07
TCGA-BH-A1FL-01A-11R-A13Q-07
TCGA-AC-A23E-01A-11R-A157-07
TCGA-AN-A0XL-01A-11R-A10J-07
TCGA-BH-A0HA-01A-11R-A12P-07
TCGA-AR-A0TR-01A-11R-A084-07
TCGA-BH-A0BZ-11A-61R-A12P-07
TCGA-A2-A04Q-01A-21R-A034-07
TCGA-BH-A0DV-01A-21R-A12P-07
TCGA-AN-A0XP-01A-11R-A109-07
TCGA-E9-A5UO-0

EDWIN: from here, call MergeHTSeqCounts... it may be tedious when there are thousands of files, but we gotta drink our own champaigne!

In [ ]:
# import gp
# # Create a GenePattern server proxy instance
# gpserver = gp.GPServer('http://gp-beta-ami.genepattern.org/gp','edwin', 'nada')
# module = gp.GPTask(gpserver, "MergeHTSeqCounts")  # Obtaining GPTask by module name
# module.param_load()

In [ ]:
# params_list = module.get_parameters()  # Get the list of GPParam objects

# for param in params_list:  # Loop through each parameter
#     print( param.get_name() )  # Print the parameter's name
#     print( param.get_default_value() )  # Print the parameter's default value
#     print( param.is_optional() )  # Print whether the parameter is optional
#     print('---')

In [7]:
# Requires GenePattern Notebook: pip install genepattern-notebook
import gp
import genepattern

# Username and password removed for security reasons.
genepattern.GPAuthWidget(genepattern.register_session("https://gp-beta-ami.genepattern.org/gp", "", ""))

GPAuthWidget()

In [22]:
mergehtseqcounts_task = gp.GPTask(genepattern.get_session(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00354')
mergehtseqcounts_job_spec = mergehtseqcounts_task.make_job_spec()
mergehtseqcounts_job_spec.set_parameter("input.files", ["https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run744999004333385378.tmp/TCGA-A2-A04Q-01A-21R-A034-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run13359705627720104.tmp/TCGA-A8-A0A7-01A-11R-A00Z-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run8046244413214232262.tmp/TCGA-A8-A07L-01A-11R-A00Z-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run1815492277280923648.tmp/TCGA-A8-A07U-01A-11R-A034-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run8566581735785711420.tmp/TCGA-A8-A09X-01A-11R-A00Z-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run708642287096279592.tmp/TCGA-A8-A085-01A-11R-A00Z-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run5108217866746811875.tmp/TCGA-AC-A6NO-01A-12R-A33J-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run5525561448153922841.tmp/TCGA-AC-A23E-01A-11R-A157-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run6881398417738258811.tmp/TCGA-AN-A0AT-01A-11R-A034-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run96581383134467718.tmp/TCGA-AN-A0FT-01A-11R-A034-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run8859801058352740992.tmp/TCGA-AN-A0XL-01A-11R-A10J-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run6726368194824000435.tmp/TCGA-AN-A0XP-01A-11R-A109-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run4134175271211784535.tmp/TCGA-AO-A0JJ-01A-11R-A056-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run489231258566206083.tmp/TCGA-AO-A1KQ-01A-11R-A13Q-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run2216797091045340775.tmp/TCGA-AO-A129-01A-21R-A10J-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run3377540792599183536.tmp/TCGA-AQ-A0Y5-01A-11R-A14M-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run8674684449151513821.tmp/TCGA-AR-A0TR-01A-11R-A084-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run3131762398876793210.tmp/TCGA-B6-A0IO-01A-11R-A034-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run1138200707289750170.tmp/TCGA-B6-A0WW-01A-11R-A109-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run909410473799098796.tmp/TCGA-B6-A0X4-01A-11R-A109-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run4995139279996150752.tmp/TCGA-BH-A0B2-01A-11R-A10J-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run7823994606100652496.tmp/TCGA-BH-A0B9-01A-11R-A056-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run5278168448293517735.tmp/TCGA-BH-A0BT-11A-21R-A12P-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run7158251054047740161.tmp/TCGA-BH-A0BZ-11A-61R-A12P-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run251543390054019885.tmp/TCGA-BH-A0DV-01A-21R-A12P-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run9099023325297799493.tmp/TCGA-BH-A0E0-01A-11R-A056-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run6171372360944734410.tmp/TCGA-BH-A0H5-11A-62R-A115-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run6681932094785163130.tmp/TCGA-BH-A0HA-01A-11R-A12P-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run6982389362884313668.tmp/TCGA-BH-A0HA-11A-31R-A12P-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run8110037704171936424.tmp/TCGA-BH-A0HI-01A-11R-A084-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run8637890844781678623.tmp/TCGA-BH-A0HQ-01A-11R-A034-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run8371416600680348057.tmp/TCGA-BH-A1EO-01A-11R-A137-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run5408216465586429843.tmp/TCGA-BH-A1FL-01A-11R-A13Q-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run2653632245583528734.tmp/TCGA-BH-A28Q-01A-11R-A16F-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run1989278716465793275.tmp/TCGA-C8-A12Y-01A-11R-A12D-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run7295390436346355757.tmp/TCGA-D8-A1J9-01A-11R-A13Q-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run5927131855017321442.tmp/TCGA-D8-A1JE-01A-11R-A13Q-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run4929899175960107598.tmp/TCGA-D8-A1JM-01A-11R-A13Q-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run2613812816244776791.tmp/TCGA-D8-A1JS-01A-11R-A13Q-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run5195433822282999316.tmp/TCGA-D8-A1XO-01A-11R-A14M-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run7491736768668996291.tmp/TCGA-E2-A2P5-01A-11R-A19W-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run4134909931064155300.tmp/TCGA-E2-A15E-06A-11R-A12D-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run7058839571208294750.tmp/TCGA-E2-A56Z-01A-12R-A29R-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run8168782250641477633.tmp/TCGA-E9-A5UO-01A-11R-A28M-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run586389469448663863.tmp/TCGA-E9-A54Y-01A-11R-A466-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run5855218547363989545.tmp/TCGA-EW-A3E8-01B-11R-A24H-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run2886770483381485156.tmp/TCGA-EW-A424-01A-11R-A24H-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run3591052746783785278.tmp/TCGA-OL-A5RZ-01A-11R-A28M-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run4874550250610649929.tmp/TCGA-PL-A8LV-01A-21R-A41B-07.htseq.counts", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run7669858897991795190.tmp/TCGA-V7-A7HQ-01A-11R-A33J-07.htseq.counts"])
mergehtseqcounts_job_spec.set_parameter("output.prefix", "<input.files_basename>")
mergehtseqcounts_job_spec.set_parameter("sampleinfo.file", "https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run7095681675137959019.tmp/TCGA_BRCA_all.txt")
mergehtseqcounts_job_spec.set_parameter("filenames.column", "0")
mergehtseqcounts_job_spec.set_parameter("class.division.column", "1")
mergehtseqcounts_job_spec.set_parameter("sample.name.column", "2")
genepattern.GPTaskWidget(mergehtseqcounts_task)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00354')

In [27]:
job9268 = gp.GPJob(genepattern.get_session(0), 9268)
genepattern.GPJobWidget(job9268)

GPJobWidget(job_number=9268)

In [6]:
import os 
from gp.data import GCT
global my_local_url
@genepattern.build_ui( name="Strip Ensembl Version and write a new GCT",
                      description="Strip out the version from the Ensembl ids in a gct file and save it as "
                      +" a new gct file on the GenePattern server.  Returns the URL to be used in the next job "
                      +"and also writes it to a notebook variable called \"my_local_url\". The method requires the gct "
                      +" file from a comlpeted MergeHTSeqCounts job.", parameters={
    "MergeHTSeqCounts_gct_file": {
        "type": "file",
        "kinds": ["gct"]
    }
})
def stripEnsembleIdAndGetLocalUrl(MergeHTSeqCounts_gct_file):
    
    
    # get the input filename and job number
    jobNum = MergeHTSeqCounts_gct_file.split("/")[-2]
    input_gct_file_Name = MergeHTSeqCounts_gct_file.split("/")[-1]
    output_gct_filename = input_gct_file_Name.replace('gct',"_versionless.gct")
    
    # get the GenePattern input job object and my username
    lastJob = gp.GPJob(genepattern.get_session(0), jobNum)
    myUserId = genepattern.get_session(0).username
    
    # this is the part that actually removes the version id
    input_gct = GCT(lastJob.get_file(input_gct_file_Name))
    df2 = input_gct.dataframe.reset_index()
    df2['Name'] = df2['Name'].apply(lambda x: x.split(".")[0])
    
    # reset the index on name and Description in case we want to look at this dataframe later
    #df2.set_index(['Name', 'Description'])
    
    # now save it back as a new file local to the Notebook server
    with open(output_gct_filename, 'w') as f:
        f.writelines('#1.2\n{}\t{}\n'.format(df2.shape[0], df2.shape[1] - 2))
        df2.to_csv(f, sep='\t', index= False)

    # upload the local file onto the GenePattern server so we can use it in the next module
    uploaded = genepattern.get_session(0).upload_file(output_gct_filename, output_gct_filename) 
    my_local_url = uploaded.get_url()  
    print("Stripped GCT file url is: "+ my_local_url)
    return my_local_url

GPUIBuilder(description='Strip out the version from the Ensembl ids in a gct file and save it as  a new gct file on the GenePattern server.  Returns the URL to be used in the next job and also writes it to a notebook variable called "my_local_url". The method requires the gct  file from a comlpeted MergeHTSeqCounts job.', function_import='stripEnsembleIdAndGetLocalUrl', name='Strip Ensembl Version and write a new GCT', params=[{'name': 'MergeHTSeqCounts_gct_file', 'label': 'MergeHTSeqCounts_gct_file', 'optional': False, 'default': '', 'description': '', 'hide': False, 'type': 'file', 'kinds': ['gct'], 'choices': []}])

In [37]:
unversioned_url = stripEnsembleIdAndGetLocalUrl(MergeHTSeqCounts_gct_file="https://gp-beta-ami.genepattern.org/gp/jobResults/9268/input.files.list.gct")

/Users/edjuaro/miniconda3/lib/python3.6/site-packages/gp/data.py:163: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.dataframe = df


Stripped GCT file url is: https://gp-beta-ami.genepattern.org/gp/users/edjuaro/tmp/run8879382091262554301.tmp/input.files.list._versionless.gct


In [10]:
collapsedataset_task = gp.GPTask(genepattern.get_session(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00134')
collapsedataset_job_spec = collapsedataset_task.make_job_spec()
collapsedataset_job_spec.set_parameter("dataset.file", "{{unversioned_url}}")
collapsedataset_job_spec.set_parameter("chip.platform", "ftp://ftp.broadinstitute.org/pub/gsea/annotations/ENSEMBL_human_gene.chip")
collapsedataset_job_spec.set_parameter("collapse.mode", "Maximum")
collapsedataset_job_spec.set_parameter("output.file.name", "<dataset.file_basename>.collapsed")
genepattern.GPTaskWidget(collapsedataset_task)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00134')

In [38]:
job9269 = gp.GPJob(genepattern.get_session(0), 9269)
genepattern.GPJobWidget(job9269)

GPJobWidget(job_number=9269)